# Prediction demonstration

1. To predict a given fluorophore, place the molecule's name (or ID) and its SMILES into the excel `To_Predict.csv` (within the data folder) following the template. Add more lines to predict more molecules together.
2. Start this notebook afresh and run all cells 
3. The predictions made are then saved to `Predictions.csv` in this folder.

In [1]:
# import required packages
import pandas as pd
from rdkit import Chem
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
tf.random.set_seed(42)
import deepchem as dc 
from scripts.graphConvModel import GraphConvModel

In [2]:
# reload model
current_tasks = ["Eg"]
model = GraphConvModel(n_tasks = len(current_tasks),
                       graph_conv_layers = [512, 512, 512, 512], 
                       dense_layers = [128, 128, 128],
                       dropout = 0.01,
                       learning_rate = 0.001,
                       batch_size = 10,
                       uncertainty = False,
                       model_dir = "Trained_Model") # <-- Trained model directory
model.restore(model.get_checkpoints()[-1])

# featurizing structures from SMILES 
df = pd.read_csv("data/To_Predict.csv")
df["MOLS"] = [Chem.MolFromSmiles(smi) for smi in df.SMILES]
graph_featurizer = dc.feat.graph_features.ConvMolFeaturizer()
graph_list = graph_featurizer.featurize(df["MOLS"])
dataset = dc.data.NumpyDataset(graph_list, ids = df["Name"])

# make predictions and save to output csv
pred = model.predict(dataset)
df["Predicted Eg (eV)"] = pred
df[["Name", "SMILES", "Predicted Eg (eV)"]].to_csv("Predictions.csv")

# print out predictions
print("Predicted energy gaps (eV)")
for i, x in enumerate(pred):
    p = str(round(x[0],4))
    print("{:<10} | {:<5}".format(df['Name'].values[i], p))

Predicted energy gaps (eV)
NDAD_4185  | 0.8881
NDAD_659   | 0.8858
NDAD_2873  | 0.9113
NDAD_4942  | 0.9211
NDAD_4204  | 0.9457
NDAD_2523  | 1.1742
NDAD_2656  | 1.0003
NDAD_442   | 1.3236
NDAD_2082  | 1.4117
NDAD_2657  | 1.0759
NDAD_1181  | 1.1903
NDAD_443   | 1.2256
NDAD_4813  | 1.2988
NDAD_4731  | 1.3002
NDAD_4936  | 1.3065
NDAD_2650  | 1.2719
NDAD_2691  | 1.3619
NDAD_436   | 1.3273
NDAD_4609  | 1.4016
NDAD_2641  | 1.4032
NDAD_2723  | 1.4071
NDA_671    | 1.042
NDA_1499   | 1.0492
NDA_3155   | 1.0584
